In [ ]:
#prepare cnn model for prediction of lung cancer

In [1]:
#importing libraries
import numpy as np
import pandas as pd
import os #os libraary is required to use the zipped dataset
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator #preprocessing of model
from tensorflow.keras.models import Sequential #to create a stack of layers or model
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Flatten,Dense,Dropout#preparartion of convolutional model
from tensorflow.keras.callbacks import EarlyStopping
from zipfile import ZipFile #this is used to extract the zipped data into a folder


In [2]:
data_path = "lung-cancer-dataset.zip"

In [3]:
#extracting data
with ZipFile(data_path,'r') as zip:
    zip.extractall()
    print("The datset has been extracted from the zip file")

The datset has been extracted from the zip file


In [4]:
#set the image size and path
image_size = 256
batch_size = 32
df = "lung-cancer-dataset/lung_images"

In [5]:
#pre-processing our imagre (downscaling the image)
datagen = ImageDataGenerator(
    rescale = 1/255,
    validation_split = 0.2
)

In [6]:
#prepare our training
train_generator = datagen.flow_from_directory(
    df,
    target_size = (image_size,image_size),
    batch_size = batch_size,
    class_mode = "categorical",
    subset = "training",
    shuffle = True
)

Found 2402 images belonging to 3 classes.


In [7]:
#prepare our training
val_generator = datagen.flow_from_directory(
    df,
    target_size = (image_size,image_size),
    batch_size = batch_size,
    class_mode = "categorical",
    subset = "validation",
    shuffle = False
)

Found 600 images belonging to 3 classes.


In [8]:
#preapre a convolutional model
model = Sequential([
    Conv2D(32,(3,3),activation = "relu",input_shape = (image_size,image_size,3)),
    MaxPooling2D(pool_size = (2,2)),

     Conv2D(64,(3,3),activation = "relu"),
    MaxPooling2D(pool_size = (2,2)),

     Conv2D(128,(3,3),activation = "relu"),
    MaxPooling2D(pool_size = (2,2)),

    Flatten(), # it will flatten the matrix so that i can my input to hidden layer
    Dense(128,activation = "relu"), #hiddden layer with 128 neurons
    Dropout(0.5),#dropping out 50% neurons to avoid overfitting
    Dense(3,activation = "softmax")#output layer and softmax function will convert the output into probability
])


c:\Users\LENOVO\anaconda3\envs\salary_env\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [9]:
model.compile(optimizer="adam", loss="categorical_crossentropy",metrics = ["accuracy"])

In [10]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 254, 254, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 127, 127, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 125, 125, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 62, 62, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 60, 60, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 30, 30, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 115200)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    14,745,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,839,363 (56.61 MB)

 Trainable params: 14,839,363 (56.61 MB)

 Non-trainable params: 0 (0.00 B)

In [11]:
#early stopping
early_stop = EarlyStopping(monitor = "val_loss",patience = 5,restore_best_weights = True)

In [12]:
#ttrain the model
history = model.fit(
    train_generator,
    epochs = 20,
    validation_data = val_generator,
    callbacks = [early_stop]
)

c:\Users\LENOVO\anaconda3\envs\salary_env\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
76/76 ━━━━━━━━━━━━━━━━━━━━ 57s 727ms/step - accuracy: 0.5906 - loss: 1.5421 - val_accuracy: 0.8300 - val_loss: 0.4080
Epoch 2/20
76/76 ━━━━━━━━━━━━━━━━━━━━ 69s 911ms/step - accuracy: 0.7933 - loss: 0.4538 - val_accuracy: 0.8667 - val_loss: 0.3632
Epoch 3/20
76/76 ━━━━━━━━━━━━━━━━━━━━ 55s 717ms/step - accuracy: 0.8075 - loss: 0.4342 - val_accuracy: 0.8033 - val_loss: 0.5609
Epoch 4/20
76/76 ━━━━━━━━━━━━━━━━━━━━ 53s 696ms/step - accuracy: 0.8592 - loss: 0.3461 - val_accuracy: 0.8283 - val_loss: 0.6759
Epoch 5/20
76/76 ━━━━━━━━━━━━━━━━━━━━ 54s 710ms/step - accuracy: 0.8581 - loss: 0.3836 - val_accuracy: 0.8950 - val_loss: 0.2944
Epoch 6/20
76/76 ━━━━━━━━━━━━━━━━━━━━ 58s 758ms/step - accuracy: 0.9034 - loss: 0.2463 - val_accuracy: 0.8767 - val_loss: 0.3617
Epoch 7/20
76/76 ━━━━━━━━━━━━━━━━━━━━ 56s 739ms/step - accuracy: 0.8560 - loss: 0.3144 - val_accuracy: 0.8767 - val_loss: 0.3073
Epoch 8/20
76/76 ━━━━━━━━━━━━━━━━━━━━ 57s 752ms/step - accuracy: 0.8937 - loss: 0.2570 - val_accu

In [13]:
model.save("LungCancerPrediction.h5")
print("model has been saved")

model has been saved
